In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import struct
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
def read_binary(file_path):

    # Инициализация списков для основных полей
    power = []
    age = []
    coordinate_x = []
    coordinate_y = []
    angle_tetta = []
    angle_phi = []
    energy = []
    time = []

    # Дополнительные важные параметры
    primary_particle = []  # PART0 (14 - p, 5626 - Fe)
    primary_energy = []    # E0 в ГэВ
    n_muons = []          # NMU (число мюонов в ШАЛ)
    n_hadrons = []        # NHADR (число адронов в ШАЛ)
    first_interaction_height = []  # H1INT (высота 1 взаимодействия в см)

    with open(file_path, 'rb') as binary_file:
        while True:
            try:
                # Пропускаем первые 3 поля (N_event, NRUN, NEVENT)
                binary_file.read(4 * 3)

                # Читаем тип первичной частицы (14 - p, 5626 - Fe)
                part0 = struct.unpack('f', binary_file.read(4))[0]
                primary_particle.append(part0)

                # Читаем энергию первичной частицы (ГэВ)
                e0 = struct.unpack('f', binary_file.read(4))[0]
                primary_energy.append(e0)

                # Чтение зенитного угла
                tetta = struct.unpack('f', binary_file.read(4))[0]
                angle_tetta.append(tetta)

                # Чтение азимутального угла
                phi = struct.unpack('f', binary_file.read(4))[0]
                angle_phi.append(phi)

                # Чтение координат оси ШАЛ
                x0 = struct.unpack('f', binary_file.read(4))[0]
                coordinate_x.append(x0)

                y0 = struct.unpack('f', binary_file.read(4))[0]
                coordinate_y.append(y0)

                # Чтение высоты первого взаимодействия
                h1int = struct.unpack('f', binary_file.read(4))[0]
                first_interaction_height.append(h1int)

                # Пропускаем NGAM, NEL
                binary_file.read(4 * 2)

                # Чтение числа адронов
                nhadr = struct.unpack('f', binary_file.read(4))[0]
                n_hadrons.append(nhadr)

                # Чтение числа мюонов
                nmu = struct.unpack('f', binary_file.read(4))[0]
                n_muons.append(nmu)

                # Чтение параметров ШАЛ
                power_eas = struct.unpack('f', binary_file.read(4))[0]
                power.append(math.log10(power_eas))

                age_eas = struct.unpack('f', binary_file.read(4))[0]
                age.append(age_eas)

                # Пропускаем NVD_edep, NVD_npe, MuBundle, MuTrackLenNVD, nMuNVD, eMuNVD, eMuNVD1
                binary_file.read(4 * 7)

                # Пропускаем новые поля 2021 (23-34) и AmplKSM[7][4][4][6] (672 значения)
                binary_file.read(4 * (12 + 672))

                # Пропускаем новые поля 2021 (707-771)
                binary_file.read(4 * 64)

                # Пропускаем EdepCntSCT[9][5][2] (90 значений)
                binary_file.read(4 * 90)

                # Чтение EdepDetNE[9][4][4] (144 значения) - используем для energy
                edep_det_ne = struct.unpack('f'*144, binary_file.read(4*144))
                energy.append(edep_det_ne)

                # Чтение TimDetNE[9][4][4][4] (576 значений) - берем пороговые времена
                tim_det_ne = struct.unpack('f'*576, binary_file.read(4*576))
                threshold_time = tim_det_ne[::4]  # Берем каждое 4-е значение (max время)
                time.append(threshold_time)

                # Пропускаем EdepStNE[9][4] (36 значений) и TimStNE[9][4][4] (144 значения)
                binary_file.read(4 * (36 + 144))

                # Чтение маркера (1762)
                binary_file.read(4)
            except struct.error:
                # Достигнут конец файла
                break

    # Создание DataFrame
    df = pd.DataFrame({
        'power': power,
        'age': age,
        'x': coordinate_x,
        'y': coordinate_y,
        'tetta': angle_tetta,
        'phi': angle_phi,
        'energy': energy,
        'threshold_time': time,
        'primary_particle': primary_particle,
        'primary_energy': primary_energy,
        'n_muons': n_muons,
        'n_hadrons': n_hadrons,
        'first_interaction_height': first_interaction_height
    })

    return df

In [ ]:
def load_and_prepare_data(data):
    static_features = data[['primary_particle', 'primary_energy', 'n_muons',
                           'n_hadrons', 'first_interaction_height']]
    sequence_energy = data[[f'energy_{i}' for i in range(144)]]
    sequence_time = data[[f'threshold_time_{i}' for i in range(144)]]

    # Извлечение целей
    targets = data[['power', 'age', 'x', 'y', 'tetta', 'phi']]

    # Преобразование phi в sin(phi) и cos(phi)
    # phi_rad = targets['phi']
    targets['sin_phi'] = np.sin(targets['phi'])
    targets['cos_phi'] = np.cos(targets['phi'])
    targets = targets.drop('phi', axis=1)

    # Разделение данных
    X_train, X_test, y_train, y_test = train_test_split(
        pd.concat([static_features, sequence_energy, sequence_time], axis=1),
        targets,
        test_size=0.2,
        random_state=42
    )

    # Препроцессинг: one-hot для primary_particle, нормализация для остальных
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(), ['primary_particle']),
            ('num', StandardScaler(), ['primary_energy', 'n_muons',
                                      'n_hadrons', 'first_interaction_height'])
        ],
        remainder='passthrough'
    )

    # Применение препроцессинга к статическим признакам
    static_cols = ['primary_particle', 'primary_energy', 'n_muons',
                  'n_hadrons', 'first_interaction_height']
    X_train_static = preprocessor.fit_transform(X_train[static_cols])
    X_test_static = preprocessor.transform(X_test[static_cols])

    # Получение имен признаков после one-hot кодирования
    cat_features = preprocessor.named_transformers_['cat'].get_feature_names_out(['primary_particle'])
    num_features = ['primary_energy', 'n_muons', 'n_hadrons', 'first_interaction_height']
    all_static_features = list(cat_features) + num_features

    # Подготовка последовательностей
    energy_cols = [f'energy_{i}' for i in range(144)]
    time_cols = [f'threshold_time_{i}' for i in range(144)]

    X_train_energy = X_train[energy_cols].values
    X_train_time = X_train[time_cols].values
    X_test_energy = X_test[energy_cols].values
    X_test_time = X_test[time_cols].values

    # Создание 3D тензоров (samples, timesteps, features)
    X_train_seq = np.stack([X_train_energy, X_train_time], axis=2)
    X_test_seq = np.stack([X_test_energy, X_test_time], axis=2)

    # Нормализация последовательностей
    seq_scaler = StandardScaler()
    X_train_seq = seq_scaler.fit_transform(
        X_train_seq.reshape(-1, X_train_seq.shape[2])
    ).reshape(X_train_seq.shape)

    X_test_seq = seq_scaler.transform(
        X_test_seq.reshape(-1, X_test_seq.shape[2])
    ).reshape(X_test_seq.shape)

    return (
        X_train_static, X_train_seq, y_train.values,
        X_test_static, X_test_seq, y_test.values,
        preprocessor, seq_scaler, all_static_features
    )

In [ ]:
os.makedirs('plots', exist_ok=True)

data = read_binary('/content/spe27p_100k_2022_correct.dat')

energy_df = pd.DataFrame(data['energy'].to_list(), columns=[f"energy_{i}" for i in range(144)])
data = pd.concat([data, energy_df], axis=1)
threshold_times_df = pd.DataFrame(data['threshold_time'].to_list(), columns=[f"threshold_time_{i}" for i in range(144)])
data = pd.concat([data, threshold_times_df], axis=1)
data = data.drop(columns = ['energy', 'threshold_time'])

# Torch

In [ ]:
class HybridDataset(Dataset):
    def __init__(self, static_features, sequence_features, targets):
        self.static_features = torch.tensor(static_features, dtype=torch.float32)
        self.sequence_features = torch.tensor(sequence_features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.static_features)

    def __getitem__(self, idx):
        return (self.static_features[idx], self.sequence_features[idx]), self.targets[idx]

In [ ]:
class HybridModel(nn.Module):
    def __init__(self, static_input_size, seq_input_size, hidden_size, num_layers, num_targets):
        super().__init__()

        # LSTM для временных рядов
        self.lstm = nn.LSTM(
            input_size=seq_input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True
        )

        # Полносвязная сеть для статических признаков
        self.static_fc = nn.Sequential(
            nn.Linear(static_input_size, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3)
        )

        # Объединенные слои
        self.combined_fc = nn.Sequential(
            nn.Linear(hidden_size * 2 + 128, 256),  # 2* для bidirectional
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3)
        )

        # Выходные головы для 7 целей (power, age, x, y, tetta, sin_phi, cos_phi)
        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 1)
            ) for _ in range(num_targets)
        ])

    def forward(self, x):
        static_input, seq_input = x

        # Обработка временных рядов
        lstm_out, _ = self.lstm(seq_input)
        seq_features = lstm_out[:, -1, :]  # Берем последний временной шаг

        # Обработка статических признаков
        static_features = self.static_fc(static_input)

        # Объединение признаков
        combined = torch.cat([static_features, seq_features], dim=1)
        features = self.combined_fc(combined)

        # Выходные головы
        outputs = [head(features) for head in self.heads]

        return torch.cat(outputs, dim=1)

In [ ]:
def train_model(model, train_loader, val_loader, num_epochs=100):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Функция потерь - MSE для всех целей
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 'min', patience=10, factor=0.5, verbose=True
    )

    train_losses, val_losses = [], []
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        # Training
        model.train()
        epoch_train_loss = 0
        for (static, seq), targets in tqdm(train_loader):
            static, seq, targets = static.to(device), seq.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model((static, seq))
            loss = criterion(outputs, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            epoch_train_loss += loss.item()

        avg_train_loss = epoch_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation
        model.eval()
        epoch_val_loss = 0
        with torch.no_grad():
            for (static, seq), targets in val_loader:
                static, seq, targets = static.to(device), seq.to(device), targets.to(device)
                outputs = model((static, seq))
                loss = criterion(outputs, targets)
                epoch_val_loss += loss.item()

        avg_val_loss = epoch_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)

        # Сохраняем лучшую модель
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            print(f"Saved new best model with val loss: {best_val_loss:.4f}")

        scheduler.step(avg_val_loss)

        print(f'Epoch {epoch+1}/{num_epochs} | '
              f'Train Loss: {avg_train_loss:.4f} | '
              f'Val Loss: {avg_val_loss:.4f} | '
              f'LR: {optimizer.param_groups[0]["lr"]:.6f}')

    # Загрузка лучшей модели
    model.load_state_dict(torch.load('best_model.pth'))

    # График обучения
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.savefig('plots/training_curve.png')
    plt.close()

    return model

# 5. Функция для восстановления phi из sin и cos
def recover_phi(sin_phi, cos_phi):
    phi_rad = np.arctan2(sin_phi, cos_phi)
    phi_deg = np.degrees(phi_rad)
    # Нормализация к диапазону [0, 360)
    phi_deg = np.where(phi_deg < 0, phi_deg + 360, phi_deg)
    return phi_deg

def evaluate_model(model, test_loader):
    device = next(model.parameters()).device
    model.eval()

    all_targets = []
    all_predictions = []

    with torch.no_grad():
        for (static, seq), targets in tqdm(test_loader):
            static, seq = static.to(device), seq.to(device)
            outputs = model((static, seq)).cpu().numpy()

            all_predictions.append(outputs)
            all_targets.append(targets.numpy())

    # Объединяем результаты
    predictions = np.vstack(all_predictions)
    targets = np.vstack(all_targets)

    # Разделяем предсказания на отдельные цели
    pred_power = predictions[:, 0]
    pred_age = predictions[:, 1]
    pred_x = predictions[:, 2]
    pred_y = predictions[:, 3]
    pred_tetta = predictions[:, 4]
    pred_sin_phi = predictions[:, 5]
    pred_cos_phi = predictions[:, 6]

    # Восстанавливаем phi
    pred_phi = recover_phi(pred_sin_phi, pred_cos_phi)

    # Аналогично для истинных значений
    true_power = targets[:, 0]
    true_age = targets[:, 1]
    true_x = targets[:, 2]
    true_y = targets[:, 3]
    true_tetta = targets[:, 4]
    true_phi = np.degrees(np.arctan2(targets[:, 5], targets[:, 6]))
    true_phi = np.where(true_phi < 0, true_phi + 360, true_phi)

    # Рассчитываем метрики для каждой цели
    metrics = {}
    target_names = ['power', 'age', 'x', 'y', 'tetta', 'phi']

    for i, name in enumerate(target_names[:5]):  # Для power, age, x, y, tetta
        true_vals = targets[:, i]
        pred_vals = predictions[:, i]

        mse = np.mean((true_vals - pred_vals) ** 2)
        rmse = np.sqrt(mse)
        mae = np.mean(np.abs(true_vals - pred_vals))
        r2 = 1 - np.sum((true_vals - pred_vals) ** 2) / np.sum((true_vals - np.mean(true_vals)) ** 2)

        metrics[name] = {
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2
        }

        # Визуализация предсказаний
        plt.figure(figsize=(10, 6))
        plt.scatter(true_vals, pred_vals, alpha=0.5)
        plt.plot([min(true_vals), max(true_vals)],
                 [min(true_vals), max(true_vals)], 'r--')
        plt.xlabel('True Values')
        plt.ylabel('Predictions')
        plt.title(f'Regression Plot for {name}')
        plt.savefig(f'plots/regression_{name}.png')
        plt.close()

    # Специальная обработка для phi
    phi_diff = np.abs(true_phi - pred_phi)
    # Учитываем циклическую природу угла
    phi_diff = np.minimum(phi_diff, 360 - phi_diff)

    metrics['phi'] = {
        'MAE': np.mean(phi_diff),
        'RMSE': np.sqrt(np.mean(phi_diff ** 2)),
        'Accuracy_5deg': np.mean(phi_diff <= 5) * 100,
        'Accuracy_10deg': np.mean(phi_diff <= 10) * 100
    }

    # Визуализация для phi
    plt.figure(figsize=(10, 6))
    plt.scatter(true_phi, pred_phi, alpha=0.5)
    plt.plot([0, 360], [0, 360], 'r--')
    plt.xlabel('True Phi (degrees)')
    plt.ylabel('Predicted Phi (degrees)')
    plt.title('Regression Plot for Phi')
    plt.savefig('plots/regression_phi.png')
    plt.close()

    # Распределение ошибок phi
    plt.figure(figsize=(10, 6))
    plt.hist(phi_diff, bins=50, alpha=0.7)
    plt.xlabel('Absolute Error (degrees)')
    plt.ylabel('Frequency')
    plt.title('Error Distribution for Phi')
    plt.savefig('plots/error_distribution_phi.png')
    plt.close()

    # Круговой график для phi
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='polar')
    ax.scatter(np.radians(true_phi), np.ones_like(true_phi),
               alpha=0.5, label='True', s=10)
    ax.scatter(np.radians(pred_phi), np.ones_like(pred_phi) * 1.1,
               alpha=0.5, label='Predicted', s=10)
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
    ax.set_rticks([])
    plt.legend()
    plt.title('Phi Predictions Comparison')
    plt.savefig('plots/polar_phi_comparison.png')
    plt.close()

    return metrics, predictions, targets

# 7. Функция для предсказания на новых данных
def predict_new_data(model, preprocessor, seq_scaler, new_data_path):
    # Загрузка новых данных
    new_data = pd.read_csv(new_data_path)

    # Извлечение признаков
    static_features = new_data[['primary_particle', 'primary_energy', 'n_muons',
                               'n_hadrons', 'first_interaction_height']]
    energy_cols = [f'energy_{i}' for i in range(144)]
    time_cols = [f'threshold_time_{i}' for i in range(144)]

    # Препроцессинг
    new_static = preprocessor.transform(static_features)

    # Подготовка последовательностей
    new_energy = new_data[energy_cols].values
    new_time = new_data[time_cols].values
    new_seq = np.stack([new_energy, new_time], axis=2)
    new_seq = seq_scaler.transform(
        new_seq.reshape(-1, new_seq.shape[2])
    ).reshape(new_seq.shape)

    # Создание DataLoader
    dataset = HybridDataset(
        new_static,
        new_seq,
        np.zeros((len(new_data), 7))  # Фиктивные цели
    )
    loader = DataLoader(dataset, batch_size=64, shuffle=False)

    # Предсказание
    device = next(model.parameters()).device
    model.eval()
    all_predictions = []

    with torch.no_grad():
        for (static, seq), _ in loader:
            static, seq = static.to(device), seq.to(device)
            outputs = model((static, seq)).cpu().numpy()
            all_predictions.append(outputs)

    predictions = np.vstack(all_predictions)

    # Восстановление phi
    pred_sin_phi = predictions[:, 5]
    pred_cos_phi = predictions[:, 6]
    pred_phi = recover_phi(pred_sin_phi, pred_cos_phi)

    # Создание DataFrame с результатами
    result_df = pd.DataFrame({
        'power': predictions[:, 0],
        'age': predictions[:, 1],
        'x': predictions[:, 2],
        'y': predictions[:, 3],
        'tetta': predictions[:, 4],
        'phi': pred_phi
    })

    return result_df

In [ ]:
BATCH_SIZE = 64
NUM_EPOCHS = 20
HIDDEN_SIZE = 256
NUM_LAYERS = 2

# 1. Загрузка данных
print("Loading and preprocessing data...")
(X_train_static, X_train_seq, y_train,
  X_test_static, X_test_seq, y_test,
  preprocessor, seq_scaler, static_feature_names) = load_and_prepare_data(data)

print(f"Static features: {len(static_feature_names)}")
print(f"Sequence shape: {X_train_seq.shape}")
print(f"Targets shape: {y_train.shape}")

# 2. Создание Dataset и DataLoader
train_dataset = HybridDataset(X_train_static, X_train_seq, y_train)
test_dataset = HybridDataset(X_test_static, X_test_seq, y_test)

# Разделение на тренировочную и валидационную выборки
train_size = int(0.85 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

Loading and preprocessing data...


/tmp/ipython-input-31-3399696855.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets['sin_phi'] = np.sin(targets['phi'])
/tmp/ipython-input-31-3399696855.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets['cos_phi'] = np.cos(targets['phi'])


Static features: 5
Sequence shape: (80000, 144, 2)
Targets shape: (80000, 7)


In [ ]:
static_input_size = X_train_static.shape[1]
seq_input_size = X_train_seq.shape[2]  # 2 признака (energy и time)
num_targets = 7  # 6 исходных целей, но phi заменен на sin и cos
model = HybridModel(
    static_input_size=static_input_size,
    seq_input_size=seq_input_size,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    num_targets=num_targets
)

print(f"\nModel architecture:\n{model}")
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")


Model architecture:
HybridModel(
  (lstm): LSTM(2, 256, num_layers=2, batch_first=True, bidirectional=True)
  (static_fc): Sequential(
    (0): Linear(in_features=5, out_features=128, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3, inplace=False)
  )
  (combined_fc): Sequential(
    (0): Linear(in_features=640, out_features=256, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.3, inplace=False)
  )
  (heads): ModuleList(
    (0-6): 7 x Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=1, bias=True)
    )
  )
)
Tota

In [ ]:
trained_model = train_model(model, train_loader, val_loader, NUM_EPOCHS)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
100%|██████████| 1063/1063 [00:58<00:00, 18.24it/s]


Saved new best model with val loss: 77.9959
Epoch 1/20 | Train Loss: 181.5760 | Val Loss: 77.9959 | LR: 0.001000


100%|██████████| 1063/1063 [00:57<00:00, 18.56it/s]


Saved new best model with val loss: 58.7914
Epoch 2/20 | Train Loss: 72.5683 | Val Loss: 58.7914 | LR: 0.001000


100%|██████████| 1063/1063 [00:57<00:00, 18.64it/s]


...

Epoch 18/20 | Train Loss: 33.9029 | Val Loss: 30.3576 | LR: 0.001000


100%|██████████| 1063/1063 [00:57<00:00, 18.64it/s]


Epoch 19/20 | Train Loss: 33.5244 | Val Loss: 31.0390 | LR: 0.001000


100%|██████████| 1063/1063 [00:56<00:00, 18.67it/s]


Saved new best model with val loss: 28.5702
Epoch 20/20 | Train Loss: 33.2199 | Val Loss: 28.5702 | LR: 0.001000


In [ ]:
# 5. Оценка
print("\nEvaluating model...")
metrics, predictions, targets = evaluate_model(trained_model, test_loader)

# Вывод метрик
print("\nPerformance Metrics:")
for target, values in metrics.items():
    print(f"\n{target}:")
    for metric, value in values.items():
        print(f"{metric}: {value:.4f}")


Evaluating model...


100%|██████████| 313/313 [00:08<00:00, 38.16it/s]



Performance Metrics:

power:
MSE: 0.1967
RMSE: 0.4435
MAE: 0.2727
R2: 0.4830

age:
MSE: 0.0041
RMSE: 0.0640
MAE: 0.0496
R2: 0.0382

x:
MSE: 83.1434
RMSE: 9.1183
MAE: 6.4189
R2: 0.8433

y:
MSE: 126.6561
RMSE: 11.2542
MAE: 7.4314
R2: 0.9173

tetta:
MSE: 20.6932
RMSE: 4.5490
MAE: 3.2865
R2: 0.8536

phi:
MAE: 89.6362
RMSE: 103.4708
Accuracy_5deg: 2.8200
Accuracy_10deg: 5.5550
